In [ ]:
#ECHURN 2.0

In [ ]:
!pip install xgboost scikit-learn pandas matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.metrics import roc_auc_score


In [ ]:
!pipdeptree

In [ ]:
#!pip install --upgrade scipy
#!pip install --upgrade numpy
!pip install numpy==1.23.0
import numpy as np
print(np.__version__)

In [ ]:
#!pip install numpy
#!pip install shap
import shap

In [ ]:
# Carregar os dados

df = pd.read_excel('Base_ECHURN_202408.xlsx')  # Substitua 'seu_arquivo.xlsx' pelo caminho para o seu arquivo Excel

#df.head()

In [ ]:
df.columns

In [ ]:
df[df['level']!=0]
df.drop (['flag_cx','desc_escolaridade','L_NPS_1','eNPS_1','var_conceito', 'num_report_total', 'tempo_promo', 'tempo_merito', 'aval_anterior'],axis=1,inplace=True)

In [ ]:
# Tratar valores nulos, se houver (exemplo: preenchendo com a média)
df.fillna(df.mean(), inplace=True)

In [ ]:
# Salvar a coluna 'nome_funcionario' em uma variável temporária
nomes_funcionarios = df['nome_funcionario']
# Excluir a coluna 'nome_funcionario' antes de aplicar o get_dummies
df_sem_nomes = df.drop(['nome_funcionario'], axis=1)
# Codificar variáveis categóricas
df_codificado = pd.get_dummies(df_sem_nomes, drop_first=True)
# Adicionar a coluna 'nome_funcionario' de volta ao DataFrame codificado
df_codificado['nome_funcionario'] = nomes_funcionarios

In [ ]:
# Separar os dados de treino e teste com base na coluna 'treino', mantendo a coluna de identificação
X_train = df_codificado[df_codificado['treino'] == 1].drop(['status', 'treino'], axis=1)
y_train = df_codificado[df_codificado['treino'] == 1]['status']

X_test = df_codificado[df_codificado['treino'] == 0].drop(['status', 'treino'], axis=1)
y_test = df_codificado[df_codificado['treino'] == 0]['status']

# Manter uma cópia do identificador dos funcionários (ex.: nome ou ID)
nomes_test = df_codificado[df_codificado['treino'] == 0]['nome_funcionario']  


In [ ]:
# Remover a coluna 'nome_funcionario' de X_train e X_test antes de treinar o modelo
X_train = X_train.drop(['nome_funcionario'], axis=1)
X_test = X_test.drop(['nome_funcionario'], axis=1)


In [ ]:
# Instanciar o modelo XGBoost
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

In [ ]:
# Construção de modelo simples
#params = {
#            'objective':'binary:logistic',
#            'learning_rate': 1.0,   # 0-1 prevenir overfitting
#            'max_depth': 4,         # Profundidade maxima por arvore
#            'subsample':1,          # % de amostras por arvore
#            'colsample_bytree':0.3, # % de cats por arvore
#            'n_estimators':100,     # num de arvores
#            'seed':42               # manter reprodutibilidade
#        }

# instantiate the classifier 
#model = XGBClassifier(**params)

In [ ]:
# Treinar o modelo
model.fit(X_train, y_train)

In [ ]:
# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]  # Probabilidades de turnover

# Mede score de acuracia
from sklearn.metrics import accuracy_score
print('XGBoost model accuracy score: {0:0.2f}'. format(
    accuracy_score(y_test, y_pred)))

In [ ]:
# Definir limiares para Baixo, Médio e Alto Risco
def classificar_risco(prob):
    if prob < 0.33:
        return 'Baixo Risco'
    elif prob < 0.67:
        return 'Médio Risco'
    else:
        return 'Alto Risco'

In [ ]:
# Aplicar a função de classificação de risco nas probabilidades
risco_turnover = [classificar_risco(prob) for prob in y_pred_prob]

# Criar um DataFrame com os resultados, incluindo o nome do funcionário e a classificação de risco
df_resultado = pd.DataFrame({
    'Nome Funcionario': nomes_test,  # Coluna de identificação
    'Probabilidade de Turnover': y_pred_prob,
    'Classificacao de Risco': risco_turnover
})

In [ ]:
# Visualizar os primeiros resultados
print(df_resultado.head())

In [ ]:
# Avaliar o modelo
# Matriz de Confusão
conf_matrix = confusion_matrix(y_test, y_pred)

# Visualizar a Matriz de Confusão
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues')
plt.title('Matriz de Confusão')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()
# Relatório de Classificação
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


In [ ]:
roc_auc = roc_auc_score(y_test, y_pred_prob)

In [ ]:
# Plotar a curva ROC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, color='blue', label=f'AUC-ROC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend()
plt.show()

In [ ]:
# make sure the SHAP values add up to marginal predictions
pred = model.predict(X_test, output_margin=True)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
np.abs(shap_values.sum(1) + explainer.expected_value - pred).max()

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")


In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
# Exportar para CSV para uma análise mais aprofundada ou relatório
#df_resultado.to_csv('resultado_turnover.csv', index=False)
df_resultado.to_excel('resultado_turnover.xlsx', index=False)

In [ ]:
# AUC-ROC
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f'\nAUC-ROC: {roc_auc:.2f}')

In [ ]:
#df[df['treino']==0]

# Separando os dados de treino e teste com base na coluna 'treino', mantendo a coluna de identificação
X_train = df[df['treino'] == 1].drop(['status', 'treino','num_span','agrupado_nivel','macro_especialidade','num_report_total','desc_escolaridade'], axis=1)
y_train = df[df['treino'] == 1]['status']

X_test = df[df['treino'] == 0].drop(['status', 'treino','num_span','agrupado_nivel','macro_especialidade','num_report_total','desc_escolaridade'], axis=1)
y_test = df[df['treino'] == 0]['status']

# Manter uma cópia do identificador dos funcionários (ex.: nome ou ID)
nomes_test = df[df['treino'] == 0]['ID']  # Substitua 'nome_funcionario' pelo nome da sua coluna de identificação

#df.drop (['treino','num_span','ID','agrupado_nivel','macro_especialidade','num_report_total','desc_escolaridade'],axis=1,inplace=True)

In [ ]:
# Tratar valores nulos, se houver (exemplo: preenchendo com a média)
df.fillna(df.mean(), inplace=True)

In [ ]:
# Codificar variáveis categóricas
df = pd.get_dummies(df, drop_first=True)

In [ ]:
# Separar variáveis independentes (X) e dependente (y)
X = df.drop('status', axis=1)  # Substitua 'alvo' pela coluna de saída (ex.: se o funcionário saiu ou não)
y = df['status']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Instanciar o modelo XGBoost
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
# Treinar o modelo
model.fit(X_train, y_train)

In [ ]:
# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]  # Probabilidades para AUC-ROC

In [ ]:
# Matriz de Confusão
conf_matrix = confusion_matrix(y_test, y_pred)


In [ ]:
# Visualizando a matriz de confusão
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues')
plt.title('Matriz de Confusão')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Relatório de classificação
print(classification_report(y_test, y_pred))

In [ ]:
# Calcular AUC-ROC
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f'AUC-ROC: {roc_auc:.2f}')

In [ ]:
# Plotar a curva ROC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, color='blue', label=f'AUC-ROC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend()
plt.show()

In [ ]:
# Definir limiares para Baixo, Médio e Alto Risco
def classificar_risco(prob):
    if prob < 0.33:
        return 'Baixo Risco'
    elif prob < 0.67:
        return 'Médio Risco'
    else:
        return 'Alto Risco'

In [ ]:
# Aplicar a função de classificação
risco_turnover = [classificar_risco(prob) for prob in y_pred_prob]

In [ ]:
# Visualizar a classificação de risco para os funcionários no conjunto de teste
df_resultado = pd.DataFrame({'Probabilidade': y_pred_prob, 'Risco de Turnover': risco_turnover})
print(df_resultado.head())